In [2]:
# <<--- FINAL DIAGNOSTIC CELL: Isko poora copy-paste karein aur run karein --- >>

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

print("--- DIAGNOSTIC TEST SHURU ---")

# 1. Hum check karenge ki kya hamari summary list sahi hai.
# Hum ek choti, nayi list bana rahe hain.
test_summaries = ['<start> this is a summary <end>', '<start> another summary <end>']
print("Test ke liye Summary List:", test_summaries)

# 2. Ab hum wahi tokenizer banayenge jo humne pehle banaya tha
print("\nTokenizer ko sahi filter ke saath bana rahe hain...")
new_filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
summary_tokenizer_test = Tokenizer(filters=new_filters)

# 3. Is naye tokenizer ko hamari test list par fit karenge
summary_tokenizer_test.fit_on_texts(test_summaries)

# 4. Ab sachai ka test!
print("\n------------------ FINAL SANITY CHECK ------------------")
print("Kya '<start>' token dictionary mein hai? --->", '<start>' in summary_tokenizer_test.word_index)
print("------------------------------------------------------")

# 5. POORA SABOOT: Hum poori dictionary hi print karke dekhenge
print("\nTokenizer ki poori dictionary (word_index):")
print(summary_tokenizer_test.word_index)
print("------------------------------------------------------")

--- DIAGNOSTIC TEST SHURU ---
Test ke liye Summary List: ['<start> this is a summary <end>', '<start> another summary <end>']

Tokenizer ko sahi filter ke saath bana rahe hain...

------------------ FINAL SANITY CHECK ------------------
Kya '<start>' token dictionary mein hai? ---> True
------------------------------------------------------

Tokenizer ki poori dictionary (word_index):
{'<start>': 1, 'summary': 2, '<end>': 3, 'this': 4, 'is': 5, 'a': 6, 'another': 7}
------------------------------------------------------


In [1]:
# <<--- SETUP CELL (Isko har baar session shuru hone par chalayein) --- >>

# 1. Zaroori libraries ko install karein
print("Installing required libraries...")
!pip install datasets
!pip install tensorflow

# 2. Ab saare imports yahan karein
print("\nLibraries installed. Now importing...")
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from datasets import load_dataset
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Attention
from tensorflow.keras.models import Model

print("\n Setup Complete! Ab aap apna baaki ka code run kar sakte hain.")

Installing required libraries...

Libraries installed. Now importing...

 Setup Complete! Ab aap apna baaki ka code run kar sakte hain.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define the path to your project folder in Google Drive
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/SummarizationProject/'

# Optional: Create the directory if it doesn't exist
import os
os.makedirs(DRIVE_PROJECT_PATH, exist_ok=True)

In [5]:
!pip install datasets

In [6]:
from datasets import load_dataset
cnn_dataset = load_dataset("cnn_dailymail", "3.0.0")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [7]:
print(cnn_dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [8]:
cnn_dataset['train'][0]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
subset_size = 25000
texts = cnn_dataset['train']['article'][:subset_size]
summaries = cnn_dataset['train']['highlights'][:subset_size]

In [11]:
import re
import string

def clean_text(text):
    text = str(text).lower()

    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)

    text = re.sub(r'[-()\"#/@;:<>{}`+=~|.!?,]', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))

    text = re.sub(r'\d+', '', text)

    text = ' '.join(text.split())

    return text

cleaned_texts = [clean_text(text) for text in texts]
cleaned_summaries = [clean_text(summary) for summary in summaries]

cleaned_summaries = ['<start> ' + summary + ' <end>' for summary in cleaned_summaries]

In [12]:
max_text_len = 500
max_summary_len = 80
summary_vocab_size = 5002

In [13]:
text_tokenizer = Tokenizer(num_words=15000, oov_token='<unk>')
text_tokenizer.fit_on_texts(cleaned_texts)
text_sequences = text_tokenizer.texts_to_sequences(cleaned_texts)
encoder_input_data = pad_sequences(text_sequences, maxlen=max_text_len, padding='post', truncating='post')

In [14]:
new_filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
summary_tokenizer = Tokenizer(num_words=10000, oov_token='<unk>', filters=new_filters)
summary_tokenizer.fit_on_texts(cleaned_summaries)

summary_sequences = summary_tokenizer.texts_to_sequences(cleaned_summaries)
decoder_input_data = pad_sequences(summary_sequences, maxlen=max_summary_len, padding='post', truncating='post')

In [15]:
decoder_target_data = decoder_input_data[:, 1:]

In [22]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Attention

embedding_dim = 256
latent_dim = 256
text_vocab_size = 10002
max_text_len = 500

encoderInputs = Input(shape=(max_text_len,))

encoderEmbedding = Embedding(input_dim=text_vocab_size, output_dim=embedding_dim)(encoderInputs)

encoderOutputs, state_h, state_c = LSTM(units=latent_dim, return_sequences=True, return_state=True)(encoderEmbedding)

encoder_states = [state_h, state_c]

In [23]:
decoderInputs = Input(shape=(None,))

decoder_embedding_layer = Embedding(input_dim=summary_vocab_size, output_dim=embedding_dim)
decoderEmbedding = decoder_embedding_layer(decoderInputs)

decoder_lstm = LSTM(units=latent_dim, return_sequences=True, return_state=True)

decoderOutputs, _, _ = decoder_lstm(decoderEmbedding, initial_state=encoder_states)

In [24]:
attentionResult = Attention()([decoderOutputs, encoderOutputs])
decoderConcatInput = Concatenate()([decoderOutputs, attentionResult])

In [25]:
decoderDense = Dense(summary_vocab_size, activation='softmax')
decoderOutputs = decoderDense(decoderConcatInput)

In [27]:
from tensorflow.keras.models import Model
model = Model([encoderInputs, decoderInputs], decoderOutputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(
    [encoder_input_data, decoder_input_data[:, :-1]],
    decoder_target_data,
    epochs=50,
    batch_size=32,
    validation_split=0.1
)

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node functional_2_1/embedding_2_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.12/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.12/asyncio/base_events.py", line 645, in run_forever

  File "/usr/lib/python3.12/asyncio/base_events.py", line 1999, in _run_once

  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3

In [ ]:
# --- INFERENCE MODELS BANANA ---

# 1. ENCODER MODEL
# Input: Article
# Output: Encoder ka poora output (attention ke liye) aur uska nichod (states)
encoder_model = Model(inputs=encoderInputs, outputs=[encoderOutputs, state_h, state_c])


# 2. DECODER MODEL
# Iske inputs thode alag honge

# Pehle, humein pichle step ke states ke liye placeholder (Input layers) chahiye
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Encoder ke poore output ke liye placeholder
decoder_hidden_state_input = Input(shape=(max_text_len, latent_dim))

# Ab hum pehle se train kiye hue layers ko istemal karenge
# Embedding layer
decoder_embedding_2 = decoder_embedding_layer(decoderInputs)

# LSTM layer, lekin is baar iska initial state naye placeholder se aayega
decoder_outputs_2, state_h_2, state_c_2 = decoder_lstm(decoder_embedding_2, initial_state=decoder_states_inputs)
decoder_states_2 = [state_h_2, state_c_2]

# Attention layer
attention_result_2 = Attention()([decoder_outputs_2, decoder_hidden_state_input])
decoder_concat_input_2 = Concatenate()([decoder_outputs_2, attention_result_2])

# Dense layer
decoder_outputs_2 = decoderDense(decoder_concat_input_2)

# Final Decoder Model
# Input: Pichla predict kiya hua shabd, encoder ka poora output, aur pichle states
# Output: Agle shabd ki prediction, aur naye states
decoder_model = Model(
    [decoderInputs] + [decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs_2] + decoder_states_2
)


print("Inference models taiyar hain!")
encoder_model.summary()
decoder_model.summary()

In [ ]:
# Yeh dictionary humein number se wapas shabd batayegi
reverse_summary_word_index = summary_tokenizer.index_word
summary_word_index = summary_tokenizer.word_index

def generate_summary(input_seq):
    """
    Ek article sequence lekar uski summary generate karta hai.
    """
    # 1. Encoder se article ka nichod nikalein
    # verbose=0 progress bar ko chup karata hai
    e_out, e_h, e_c = encoder_model.predict(input_seq, verbose=0)

    # 2. Summary ki shuruaat '<start>' token se karein
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = summary_word_index['<start>']

    stop_condition = False
    decoded_sentence = ''

    # 3. Loop chalao jab tak '<end>' na mil jaye ya max length poori na ho jaye
    while not stop_condition:
        # Decoder ko pichla shabd aur states dekar agla shabd predict karwayein
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c], verbose=0)

        # Sabse zyada probability wala shabd chunein
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_summary_word_index.get(sampled_token_index, '<unk>')

        # Agar model ne '<end>' bola, to ruk jao
        if (sampled_word == '<end>' or len(decoded_sentence.split()) >= (max_summary_len - 1)):
            stop_condition = True
        else:
            # Warna, summary mein naya shabd jod do
            decoded_sentence += ' ' + sampled_word

        # Agle loop ke liye taiyari: Naya predict kiya hua shabd agla input banega
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        e_h, e_c = h, c

    return decoded_sentence.strip()


# --- Chaliye, ab test karte hain! ---
print("--- MODEL TESTING ---")
# Hum apne data se koi 5 random articles uthate hain
for i in np.random.randint(0, len(encoder_input_data), 5):
    input_article = encoder_input_data[i:i+1] # Article ko model ke format mein daala
    original_article = texts[i]
    original_summary = summaries[i]
    generated_summary = generate_summary(input_article)

    print("\n-------------------------------------------------")
    print("ORIGINAL ARTICLE (Snippet):", original_article[:250] + "...")
    print("\nORIGINAL SUMMARY:", original_summary)
    print("\n>> GENERATED SUMMARY:", generated_summary)
    print("-------------------------------------------------")

In [ ]:
print("Kya '<start>' token dictionary mein hai?")
print('<start>' in summary_tokenizer.word_index)

In [22]:
# <<--- FINAL STEP: Manual Input ke liye Code --- >>

def summarize_text(raw_text):
    """
    Ek raw text string leta hai, use model ke liye taiyar karta hai,
    aur uski summary generate karke print karta hai.
    """
    # 1. Input text ko saaf karein (bilkul training data ki tarah)
    print("⏳ Text ko saaf kiya ja raha hai...")
    cleaned_input = clean_text(raw_text)

    # 2. Text ko numbers ki series mein badalein (Tokenizer ka istemal karke)
    print("⏳ Text ko numbers mein badla ja raha hai (Tokenizing)...")
    input_sequence = text_tokenizer.texts_to_sequences([cleaned_input])

    # 3. Sequence ko pad karke 'max_text_len' (e.g., 500) length ka banayein
    print("⏳ Sequence ko padding kiya ja raha hai...")
    padded_input = pad_sequences(input_sequence, maxlen=max_text_len, padding='post', truncating='post')

    # 4. Ab hamara 'generate_summary' function istemal karein jo humne pehle banaya tha
    print("🧠 Summary generate ki ja rahi hai...")
    generated_summary = generate_summary(padded_input)

    # 5. Aakhir mein, saaf-suthra result print karein
    print("\n================== FINAL SUMMARY ==================")
    print("\n📜 ORIGINAL TEXT:")
    print(raw_text)
    print("\n\n✨ GENERATED SUMMARY:")
    print(generated_summary)
    print("\n===================================================")

# --- Ab is function ko test karte hain! ---

# Yahan aap apna koi bhi text daal sakte hain.
# Bas is variable ke andar "" ke beech mein text badal dein.
my_custom_article = """
The Indian Space Research Organisation or ISRO is the national space agency of India, headquartered in Bengaluru.
It operates under the Department of Space which is directly overseen by the Prime Minister of India, while the Chairman of ISRO acts as the executive of DOS as well.
ISRO is India's primary agency for performing tasks related to space-based applications, space exploration and the development of related technologies.
It is one of six government space agencies in the world which possess full launch capabilities, deploy cryogenic engines, launch extraterrestrial missions and operate large fleets of artificial satellites.
"""

# Chaliye, is custom article ki summary banate hain
summarize_text(my_custom_article)

# Aap ek aur example bhi test kar sakte hain:
another_article = """
Artificial intelligence is intelligence demonstrated by machines, as opposed to the natural intelligence displayed by animals including humans.
Leading AI textbooks define the field as the study of "intelligent agents": any system that perceives its environment and takes actions that maximize its chance of successfully achieving its goals.
Some popular accounts use the term "artificial intelligence" to describe machines that can mimic "cognitive" functions that humans associate with the human mind,
such as "learning" and "problem solving", however this definition is rejected by major AI researchers.
"""
summarize_text(another_article)

⏳ Text ko saaf kiya ja raha hai...
⏳ Text ko numbers mein badla ja raha hai (Tokenizing)...
⏳ Sequence ko padding kiya ja raha hai...
🧠 Summary generate ki ja rahi hai...


NameError: name 'generate_summary' is not defined

In [23]:
# <<--- STEP 1: Saari Zaroori Files ko Save Karna --- >>
import pickle
from google.colab import drive
import os

print("⏳ Google Drive ko connect kiya ja raha hai...")
drive.mount('/content/drive')

# Ek naya folder banayein jahan sab kuch save hoga
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/Summarization_Model_Files/'
os.makedirs(DRIVE_PROJECT_PATH, exist_ok=True)
print(f"✅ Drive connected. Files ko '{DRIVE_PROJECT_PATH}' mein save kiya jayega.")

# --- Models ko .h5 format mein save karein ---
print("\n⏳ Models ko save kiya ja raha hai...")
model.save(os.path.join(DRIVE_PROJECT_PATH, 'full_model.h5'))
encoder_model.save(os.path.join(DRIVE_PROJECT_PATH, 'encoder_model.h5'))
decoder_model.save(os.path.join(DRIVE_PROJECT_PATH, 'decoder_model.h5'))
print("✅ 3 models (.h5) सफलतापूर्वक save ho gaye.")

# --- Tokenizers ko .pkl format mein save karein ---
print("\n⏳ Tokenizers ko .pkl format mein save kiya ja raha hai...")
with open(os.path.join(DRIVE_PROJECT_PATH, 'text_tokenizer.pkl'), 'wb') as f:
    pickle.dump(text_tokenizer, f)

with open(os.path.join(DRIVE_PROJECT_PATH, 'summary_tokenizer.pkl'), 'wb') as f:
    pickle.dump(summary_tokenizer, f)
print("✅ 2 tokenizers (.pkl) सफलतापूर्वक save ho gaye.")

print("\n\n🎉 Sab kuch Google Drive mein save ho gaya hai!")

⏳ Google Drive ko connect kiya ja raha hai...


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive connected. Files ko '/content/drive/MyDrive/Summarization_Model_Files/' mein save kiya jayega.

⏳ Models ko save kiya ja raha hai...


NameError: name 'encoder_model' is not defined